<a href="https://colab.research.google.com/github/samec-s/ownlight/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import os
import json
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Your paths (after mounting)
renders_dir = '/content/drive/MyDrive/Data/model1/renders'
labels_dir = '/content/drive/MyDrive/Data/model1/labels'

# Load data (adjust range if you have more/less than 10)
images = []
labels = []

for i in range(1, 11):  # 001 to 010 — change if needed
    png_file = f'optimal_{i:03d}.png'
    json_file = f'optimal_{i:03d}.json'

    img_path = os.path.join(renders_dir, png_file)
    json_path = os.path.join(labels_dir, json_file)

    if not os.path.exists(img_path) or not os.path.exists(json_path):
        print(f"Missing: {png_file} or {json_file}")
        continue

    # Load image
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    images.append(img_array)

    # Load label
    with open(json_path, 'r') as f:
        data = json.load(f)['optimal_params']
    label = np.array([
        data['key_azimuth'], data['key_elevation'], data['key_temp'],
        data['key_intensity'], data['key_distance'],
        data['fill_azimuth'], data['fill_elevation'], data['fill_temp'],
        data['fill_intensity'], data['fill_distance']
    ])
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

print(f"Loaded {len(images)} image-label pairs")

# Build model
base = MobileNetV3Small(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base.trainable = False

model = Sequential([
    base,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(10)  # 10 output values
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train
history = model.fit(images, labels, epochs=50, batch_size=4, validation_split=0.2)

# Save model
model.save('/content/drive/MyDrive/OwnLight_Data/own_light_mvp.h5')

print("Training done. Model saved to Drive.")

Missing: optimal_004.png or optimal_004.json
Missing: optimal_005.png or optimal_005.json
Missing: optimal_010.png or optimal_010.json
Loaded 7 image-label pairs
4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 6495237.5000 - val_loss: 8068156.0000
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 6494053.5000 - val_loss: 8066873.5000
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 6492924.0000 - val_loss: 8065799.5000
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 6204513.0000 - val_loss: 8064726.0000
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 6490967.5000 - val_loss: 8063606.5000
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 6202552.5000 - val_loss: 8062415.0000
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 6201556.0000 - val_loss: 8061154.0000
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 6487777.5000 - val_loss: 8059812.5000
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━

Training done. Model saved to Drive.
